In [ ]:
import pandas as pd
import io
import requests

Dane pochodzą z https://dane.gov.pl/pl/dataset/188,wykaz-urzedowych-nazw-miejscowosci-i-ich-czesci

In [ ]:
#ściągnięcie danych z GitHub
url='https://raw.githubusercontent.com/NXTRSS/RNNCourse/main/Wykaz_urzędowych_nazw_miejscowości_stan_na_1.1.2019.csv'
s=requests.get(url).content
df_cities=pd.read_csv(io.StringIO(s.decode('utf-8')))
df_cities

In [ ]:
df_cities['Rodzaj'].value_counts()[:5]

Każdą miejscowośc poprzedzamy tokenem sos i kończymy eos. W naszym przypadku umawiamy sie na $ i ^ - ponieważ model będzie oparty o znaki i wygodniej jet kiedy tokeny specjalne również są znakami.

In [ ]:
'''
<sos>Warszawa</eos>
$Warszawa^

$Warszawa
Warszawa^
'''

Do modelu wybieramy tylko nazwy miast

In [ ]:
list_of_cities = df_cities[df_cities['Rodzaj'] == 'miasto']['Nazwa miejscowości'].tolist()
#PROSZE UZUPEŁNIĆ: dodać znak $ przed każdym miastem i ^ na końcu (czyli chcemy aby Warszawa -> $Warszawa^)


list_of_cities[:10]


Dane będą dopełnione zerami do najdłuższej nazwy w datasecie. Powoduje to pewnien problem - krótsze nazwy będą składały się w dużej mierze z zer (paddingu). W takiej sytuacju model będzie uczył się że pad trzeba zamienić na pad. Nie jest to zła wiedza natomiast rodzi problem - zamiana pad na pad jest bardzo łatwa i nie jest nam do niczego potrzebna. Jeżeli tokeny pad będą uwzględniane przy liczneiu loss to będzie ona sztucznie zaniżona. Np. dla nazwy która w połowie składa się z paddingu, loss będzie zaniżony o połowę. Dlatego w późniejszej części ustawiamy maskowanie - tj ignorowanie klasy o indeksie 0. Nie będzie ona uwzględniana przy liczeniu wartości funkcji straty co da nam bardziej realistyczny obraz sytuacji.

```
$Baborów^0000000000000
$Baranów Sandomierski^

$Baranów Sandomierski
Baranów Sandomierski^

$Baborów000000000000
Baborów^000000000000
```




In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import TextVectorization, Embedding, LSTM, TimeDistributed, Dense, Dropout, GRU
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.activations import softmax
import tensorflow.keras as keras

In [ ]:
tokenizer = Tokenizer(filters='', lower=False, char_level=True) # resetuejmy filtr bo domyślnie usunie tokeny $ i ^, nie zmniejszamy liter - model ma się nauczyć poprawnej wielkości
#PROSZĘ UZUPEŁNIĆ: proszę wytrenować tokenizer na naszej liście miast



vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
# tokenizer.word_index

In [ ]:
tokenizer.word_index

In [ ]:
#PROSZĘ UZUPEŁNIĆ: przygotować dane X_train poprzez podane nazwy miasta bez ostatniego znaku np. $Warszawa^ -> $Warszawa
X_train = None
#przygotować dane X_train poprzez podane nazwy miasta bez pierwszego znaku np. $Warszawa^ -> Warszawa^
y_train = None

print(f'example input: {X_train[0]}, example output: {y_train[0]}')

In [ ]:
#proszę użyć tokenizera na X_train i y_train
X_train = None
y_train = None

#proszę dodać padding (jego długość ustali się automatycznie po podaniu danych), tak aby 0 były dodane na końcu sekwencji
X_train = None
y_train = None

print(f'example input: {X_train[0]}, example output: {y_train[0]}')

In [ ]:
X_train.shape

In [ ]:
class LanguageModel(keras.Model):
    def __init__(self):
        super().__init__(self)
        #proszę zadeklarować warstę embedding z odpowiednim kszatłtem, 
        #wielkością wektora 32 oraz ustawieniem parametru uwzględniającego maskę zer
        self.embedding = None

        #Proszę zadeklarować warstę GRU z 48 jednostkami oraz ze zwracaniem 
        #sekwencji i stanu ukrytego
        self.rnn = None

        #proszę zadeklarować warstę gęstą z odpowiednią liczbą wyjścia (jaka 
        #jest liczba klas?) oraz funkcją aktywacji softmax
        self.fc = keras.layers.Dense(vocab_size, activation='softmax')
    
    def call(self, inputs, training=False, hidden_state=None, return_state=False): # pamiętajcie, że domyślne wartości argumentów w call powinny być ustawione do infernecji
        """
        inputs - wejście w formacie [batch size x max seq len]
        training - czy w trakcie teningu
        hidden_state - initial state od którego sieć ma zacząć. Jeżeli None - domyślny start
        retur_state - czy zwrócić ostatni hidden state?
        """
        x = inputs
        x = self.embedding(x, training=training) # embedujemy

        if hidden_state is None: 
            hidden_state = self.rnn.get_initial_state(inputs=x) # pobieramy domyślny hidden state jeżeli nie podano żadnego. Zwykle jest to tensor zer

        seq_out, last_hidden_state = self.rnn(x, training=training, initial_state=hidden_state)
        x = self.fc(seq_out) # mamy predykcję znaku (a raczej jej rozkład)

        if return_state: # zwracamy predykcję i ostatni state
            return x, last_hidden_state
        return x #zwracamy tylko predykcję

model = LanguageModel()
model(X_train)
# model.build(input_shape=[None, 1])
model.summary()
criterion = keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer='adam', loss=criterion)




In [ ]:
# proszę wytrenować model na odpowiednich danych przez 100 epok
history = None

In [ ]:
# Próbkowanie litera po literze (znak po znaku)

prompt = "$A" # ustalamy prompt
encoded_prompt = tokenizer.texts_to_sequences([prompt]) # enkodujemy go
encoded_prompt = tf.convert_to_tensor(encoded_prompt) # i zamieniamy na tensor
hidden_state = None # pierwszy hidden state zostawiamy domyślny

for i in range(30): # nie więcej niż 30 znaków
    model_out, hidden_state = model(encoded_prompt, return_state=True, hidden_state=hidden_state) # pobieramy predykcje modelu oraz ostatni hidden state. Zwróć uwagę
    # że przekazujemy hidden state
    last_output = model_out[0, -1, :].numpy() # wybieramy predykcję ostatniego znaku
    next_char_idx = np.random.choice(np.arange(len(last_output)),  p=last_output) # i samplujemy zgodnie z rozkładem prawdopodobieństwa
    next_char = tokenizer.index_word[next_char_idx] # zamieniamy indeks znaku na znak
    prompt += next_char # i dopisujemy do prompt

    encoded_prompt = tokenizer.texts_to_sequences([next_char]) # nowe wejście do modelu składa się TYLKO z jednego, nowowygenerowanego znaku
    # ponieważ informacja o dotychczasowje sekwencji zawarta jest w hidden state którym zarządamy manualnie w przeciwieństgwie do uproszczonej metody
#     encoded_prompt = tokenizer.texts_to_sequences([prompt])
    encoded_prompt = tf.convert_to_tensor(encoded_prompt)

    if next_char == '^':
        break

prompt

In [ ]:
# próbkowanie/samplowanie modelu
# ustalamy wartość początkową (nazywaną seed/prompt/warmpup text)

prompt = "$" # tylko <sos> - niech modlel wypluje nazwe na dowolną literę

for i in range(30): # jeżeli przez 30 znaków model nie uzna za stosowne wyprodukować <EOS> oznaczający koniec, przerywamy. 
    encoded_prompt = tokenizer.texts_to_sequences([prompt]) # enkodujemy prompt
    output = model.predict(encoded_prompt) # generujemy predykcję znaków
    last_output = output[0, -1, :] # interesuje nas tylko ostatni. Zwróć uwage że operujemy na batch_size=1
    # next_char_idx = last_output.argmax() - greedy sampling - wybierz zawsze najbardziej prawdopodbny znak. W takim scenariuszu generowane nazwy będą zawsze takie same!
    next_char_idx = np.random.choice(np.arange(len(last_output)),  p=last_output) # propabilistic samplig - wybierz znak zgodnie z rozkładem prawdopodobieństwa zwróconym przez model
    # jest jeszcze beam search - najbardziej zaawansowana metoda której nie omawialiśmy
    next_char = tokenizer.index_word[next_char_idx] # zamień indeks znaku na konkretny znak
    prompt += next_char # i dopisz go do prompt, 

    if next_char == '^': # jeżeli model uznał że skończył to kończymy samplowanie
        break

print(prompt)


In [ ]:
np.random.choice(np.arange(len(last_output)),  p=last_output)

In [ ]:
for c, i in tokenizer.word_index.items():
    prob = last_output[i]
    if prob > 0.001:
        print(f'{c}:{prob:.3f}')

```
X = Warszaw (a)

y = model(X)

a - 0.75
  - 0.1
- - 0.1

X2 = Warszawa
y2 = model(X2) -> ^

X = $ y = W   O(1)
$W, a         O(2)
$Wa, r        O(3)

...



```


